# Project 1: Vacuum Cleaner Agent
## Author: Your Name Here

### Purpose:
Invesitgate the creation, application and assesment of an agent program.

## Assignment:
Create vacuum cleaner environment which clearly separates the sensor, actuators and environment from the agent portion of the code (Look at the blind dog problem, remember a hungry dog is like a vacuum cleaner).

The environment is a 10x10 square room with a given probalibity of dirt appearing in any square specificed.  The robot vacuum starts in a random location and cleans for a given number of time steps.  The performance measure is given by portion of the dirt cleaned (amount of dirt removed divided by the amount of dirt at the start).  To get an average performance for an agent for a given probabilty of dirt appearing run the simulation 200 times and average.   Repeat for the following dirt probabilities (10%, 20% and 30%).

Create the following types of agents and measure the performance of each.  Explain your results.

- Simple (deterministic) reflex agent (just sense, act, sense, act), the agent has a dirt sensor and a bump sensor. It can clean, turn (left or right), and move forward.

- Simple (random) reflex agent, as above but can take a random action.

- A reflex agent which is deterministic but can remember the last 4 sensor inputs and last 4 actions.


Documentation must include:
- Introduction
- Discussion of Solution
- Testing Design and Expected Results
- Actual Results
- Discussion of Results
- Conclusion
- Code provided in Azure notebook that can be used to duplicate testing.


In [1]:
!pip install ipythonblocks

In [11]:
import sys
sys.path.append('/home/nbuser/library/')
# This is used to include files which are in the same folder.

from utils import *
from agents import * 


# agents.py must be in the same folder as the notebook
# utils.py must be in the same folder as the notebook

class GraphicRoom(GraphicEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        super().__init__(width, height, boundary, color, display)
        
    def thing_classes(self):
        return [Wall, Dirt, Bump, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]    
        
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        # This bump was from the last move, so remove it
        items = self.list_things_at(agent.location, tclass=Bump)
        if len(items) != 0:
            self.delete_thing(items[0])
            
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
                agent.location = loc  # We can move forward so update it...the vacuum doesn't know anything
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                self.add_thing(Bump(),agent.location)
                print(self.list_things_at(agent.location))
                agent.moveforward(False)
               
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
                    
    def is_done(self):
       
        return False

class Dirt(Thing):
    pass

class Bump(Thing):
    pass

class Vacuum(Agent):
    
    direction = Direction("down")
        
    def clean(self, thing):
        print("Vacuum Cleaned Location {}.".format(self.location))
            
    def moveforward(self, success=True):
        print("Vacuum Moved Forward {}.".format( self.location))
        
    def turn(self, direction):
        print("Vacuum turned")
        self.direction = self.direction + direction

def program(percepts):
    print(percepts)
    for p in percepts:
        if isinstance(p, Dirt):
            return('clean')
        if isinstance(p, Bump):
            return('turnleft')
    return('moveforward')

In [12]:
from random import randint

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(program)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(5)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [10, 10], but couldn't
[<Vacuum>, <Bump>]
Vacuum Moved Forward [10, 10].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>, <Bump>]
Vacuum decided to turnleft at location: [10, 10]
Vacuum turned


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>, <Bump>]
Vacuum decided to turnleft at location: [10, 10]
Vacuum turned


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move upwards at location: [10, 10]
Vacuum Moved Forward [10, 10].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


In [3]:
vac.location

[5, 10]

In [13]:
def randomprogram(percepts):
    print(percepts)
    actions = ['turnleft', 'turnright', 'moveforward', 'moveforward']
    for p in percepts:
        if isinstance(p, Dirt):
            return('clean')
        if isinstance(p, Bump):
            return(actions[randint(0,1)])
    return(actions[randint(0,3)])

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(randomprogram)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(5)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to turnleft at location: [1, 9]
Vacuum turned


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move rightwards at location: [1, 9]
Vacuum Moved Forward [1, 9].


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [2, 9]
Vacuum Moved Forward [2, 9].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [2, 10], but couldn't
[<Vacuum>, <Bump>]
Vacuum Moved Forward [2, 10].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


In [17]:
memory = ['moveforward', 'moveforward', 'moveforward', 'moveforward']
def memprogram(percepts):
    global memory
    def updatemem(move, memory):
        memory.pop(0)
        memory.append(move)
        return memory
    print(percepts)
    actions = ['turnleft', 'turnright', 'moveforward', 'moveforward']
    if memory == ['moveforward', 'moveforward', 'turnleft', 'moveforward']:
        memory = updatemem('turnleft', memory)
        return 'turnleft'
    if memory == ['moveforward', 'moveforward', 'turnright', 'moveforward']:
        memory = updatemem('turnright', memory)
        return 'turnright'
    for p in percepts:
        if isinstance(p, Dirt):
            return('clean')
        if isinstance(p, Bump):
            x = actions[randint(0,1)]
            memory = updatemem(x, memory)
            return x
    memory = updatemem('moveforward', memory)
    return 'moveforward'

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(memprogram)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(20)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [6, 8]
Vacuum Moved Forward [6, 8].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [6, 9]
Vacuum Moved Forward [6, 9].


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to turnright at location: [8, 1]
Vacuum turned


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


[<Vacuum>]
Vacuum decided to move downwards at location: [8, 1]
Vacuum Moved Forward [8, 1].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
